#### Functions (IGNORE)

In [1]:
import yfinance as yf
missing_data_tickers = [] # use this as a list of tickers with missing data

def get_data_from_start_to_end(ticker, start_date, end_date):
    global missing_data_tickers  # Use the global list to accumulate missing tickers
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if stock_data.empty:
            missing_data_tickers.append(ticker)
            raise ValueError(f"Stock data for ticker {ticker} during the period from {start_date} to {end_date} was not found.")
        return stock_data
    except Exception as e:
        print(f"An error occurred for ticker {ticker}: {e}")
        missing_data_tickers.append(ticker)
        return None


In [2]:
# for a variety of periods load in different list of tickers
def download_stock_data_for_periods(tickers, periods):
    all_data = {}
    
    for period, (start_date, end_date) in periods.items():
        period_data = {}
        for ticker in tickers:
            data = get_data_from_start_to_end(ticker, start_date, end_date)
            if data is not None:
                period_data[ticker] = data
        all_data[period] = period_data
    
    return all_data

In [3]:
import pandas as pd

# Get the adjusted close prices
adj_close_sector_etf = {}

# Create adjusted close price only listing of sector ETFs
def get_adjusted_closed_price(nested_dict, tickers, periods):
    for period in periods:
        stock_price_df = pd.DataFrame()  # Create a new DataFrame for each period
        for ticker in tickers:
            stock_price_df[ticker] = nested_dict[period][ticker]['Adj Close']
        
        adj_close_sector_etf[period] = stock_price_df  # Store the complete DataFrame for the period
    
    return adj_close_sector_etf

In [4]:
import random

def stochastic_modeling(nested_dict, tickers, periods,num_samples):
    # Store the returns in a nested dictionary
    nested_dict_returns = {period: {ticker: [] for ticker in tickers} for period in periods}

    # Go through each economic time period
    for period in periods:
        max_index = len(nested_dict[period]) - 90  # Ensure there's enough data to calculate ROI

        # Generate random samples from the valid range
        random_dates = random.choices(range(max_index), k=num_samples)

        for ticker in tickers:
            for date_idx in random_dates:
                start_price = nested_dict[period][ticker].iloc[date_idx]
                end_price = nested_dict[period][ticker].iloc[date_idx + 90]

                # Get the return by the Holding Period Return
                roi = (((end_price - start_price) / start_price) * 100)

                nested_dict_returns[period][ticker].append(roi)

    return nested_dict_returns  # Return the nested dictionary with returns

In [5]:
def stochastic_roi(tickers,periods,return_rates_list,analysis_type):
    df = pd.DataFrame(index=tickers,columns=periods)
    for period in periods:
        for ticker in tickers:
            data = pd.Series(return_rates_list[period][ticker])
            if analysis_type=='Mean':
                df.at[ticker,period] = data.mean()
            elif analysis_type=='Median':
                df.at[ticker,period] = data.median()
            elif analysis_type=='Std':
                df.at[ticker,period] = data.std()
            elif analysis_type=='Variance':
                df.at[ticker,period] = data.var()

    return df

# Technical Analysis Investment Strategy

In [6]:
# import packages
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
# create time periods for where this takes place
economic_cycle_periods = {

    "trough": ("2008-10-01", "2009-06-01"),
    "expansion": ("2012-01-01", "2015-01-01"),
    "peak": ("2019-06-01", "2020-02-01"),
    "contraction": ("2007-12-01", "2008-10-01"),
}

economic_cycle_periods_list = ['trough','expansion','peak','contraction']

In [8]:
# create etf tickers for sectors
sector_etf_tickers = [
    'XLB', # materials sector
    'XLI', # industrials sector
    'XLF', # financials
    'XLK', # information technology
    'XLY', # consumer discretionary
    'XLP', # consumer staples
    'XLE', # energy
    'XLV', # healthcare
    'VOX', # communication services
    'XLU', # utilities
    'IYR' # real estate
    ]

## Buy and Hold Investment Technique
The buy and hold strategy is a passive investing strategy that will be applied to the 11 sector ETFs during different macroeconomic time periods.

In [9]:
# save nested dictionary data as a variable to be accessed.
sector_etf_data = download_stock_data_for_periods(sector_etf_tickers,economic_cycle_periods)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [10]:
# get adjusted close price
sector_etf_adjusted_close = get_adjusted_closed_price(sector_etf_data,sector_etf_tickers,economic_cycle_periods_list)

### Perform stochastic modeling using buy and hold strategy
Use a different day where the stock begins investing then hold for a month and see the return.

In [11]:
# perform stochastic modeling on the buy and 
stochastic_buy_hold = stochastic_modeling(sector_etf_adjusted_close,sector_etf_tickers,economic_cycle_periods_list,10000)

In [12]:
# this can be repeated for mean, median, std and var
stochastic_roi(sector_etf_tickers,economic_cycle_periods_list,stochastic_buy_hold,'Mean')

,trough,expansion,peak,contraction
XLB,-0.810123,4.939765,4.12702,-0.471373
XLI,-13.057645,6.413633,6.865773,-4.652696
XLF,-21.567214,7.277606,9.60327,-15.684469
XLK,3.525604,5.614588,12.613565,-2.878304
XLY,-0.952595,7.024736,3.426692,-4.365587
XLP,-8.117934,5.748044,5.210943,0.482554
XLE,-4.632107,3.687888,-0.386603,3.408302
XLV,-4.700696,9.078469,8.477328,-3.778369
VOX,4.250258,5.448187,6.128374,-5.550549
XLU,-8.699519,4.869357,6.325401,-2.701162


## Bollinger Bands Investment Technique
Using John Bollinger's techniques 'Bollinger Bands' to create buy and sell signals to observe the roi for investing for a month.

In [13]:
# add bollinger data
import scipy.stats as stats
def add_bollinger_data(data,window,conf_int):
        z_score = stats.norm.ppf(1 - (1 - conf_int) / 2) # create a zscore from the mean

        data['middle_band'] = data['Adj Close'].rolling(window).mean()
        data['upper_band'] = data['middle_band'] + z_score * data['Adj Close'].rolling(window).std()
        data['lower_band'] = data['middle_band'] - z_score * data['Adj Close'].rolling(window).std()

        data['Signal'] = None

        data['Signal'] = np.where(data['Adj Close'] < data['lower_band'], 'Buy', 
                              np.where(data['Adj Close'] > data['upper_band'], 'Sell', np.nan))

        return data

In [14]:
# create bollinger data for multiple time period and multiple tickers
def bollinger_data_multiple_periods_tickers(periods,tickers,data,window,confidence_period):
    # for each ticker in economic time periods
    for period in periods:
            for ticker in tickers:
                    try:
                        add_bollinger_data(data[period][ticker],window,confidence_period)
                    except KeyError:
                        print(f'Data for {ticker} does not exist during {period}')

In [15]:
# create bollinger bands in stock data
bollinger_data_multiple_periods_tickers(economic_cycle_periods_list,sector_etf_tickers,sector_etf_data,20,0.95)
sector_etf_data['trough']['XLB']

,Open,High,Low,Close,Adj Close,Volume,middle_band,upper_band,lower_band,Signal
Date,,,,,,,,,,
2008-10-01,32.759998,33.189999,32.130001,32.849998,23.119268,14639500,NaN,NaN,NaN,nan
2008-10-02,31.540001,31.860001,29.930000,30.490000,21.458340,12581300,NaN,NaN,NaN,nan
2008-10-03,30.190001,31.690001,29.780001,30.190001,21.247204,16770600,NaN,NaN,NaN,nan
2008-10-06,29.510000,29.510000,26.889999,28.700001,20.198563,22512700,NaN,NaN,NaN,nan
2008-10-07,29.160000,29.530001,27.049999,27.219999,19.156965,16004900,NaN,NaN,NaN,nan
...,...,...,...,...,...,...,...,...,...,...
2009-05-22,26.530001,26.660000,26.110001,26.299999,18.848713,8421500,18.742288,19.907895,17.576680,nan
2009-05-26,26.170000,26.969999,25.830000,26.930000,19.300222,7886900,18.827573,19.888618,17.766528,nan
2009-05-27,26.790001,26.850000,25.860001,25.920000,18.576374,7216600,18.881324,19.759551,18.003097,nan


In [16]:
# example case of bollinger bands in stock data
sector_etf_data['expansion']['XLB']['Signal'].value_counts()

nan     670
Buy      45
Sell     39
Name: Signal, dtype: int64

In [17]:
def collect_signals(nested_dict, periods, tickers):
    # Initialize an empty dictionary to hold DataFrames for each period
    bb_nested_dict = {}

    for period in periods:
        # Create a DataFrame for each period with the tickers as columns
        signals_period = pd.DataFrame(columns=tickers)
        
        # Loop through each ticker and extract the 'Signal'
        for ticker in tickers:
            signals_period[ticker] = nested_dict[period][ticker]['Signal']
        
        # Store the DataFrame in the dictionary using the period as the key
        bb_nested_dict[period] = signals_period

    # Return the dictionary containing DataFrames for each period
    return bb_nested_dict

In [18]:
bb_bands_signals = collect_signals(sector_etf_data,economic_cycle_periods_list,sector_etf_tickers)
bb_bands_signals['trough']

,XLB,XLI,XLF,XLK,XLY,XLP,XLE,XLV,VOX,XLU,IYR
Date,,,,,,,,,,,
2008-10-01,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2008-10-02,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2008-10-03,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2008-10-06,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2008-10-07,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...
2009-05-22,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2009-05-26,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2009-05-27,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [19]:
# make sure that the length of the two dataframes are same
# this is so that you can treat them as two panes stacked on top of one another and each index is relevant to the date and ticker
len(sector_etf_adjusted_close) == len(bb_bands_signals)

True

In [20]:
from datetime import timedelta

# goal is to create a function that uses the bb signal df and adj close signal df
# you run through the signal day by day ticker by ticker
# get the buy/sell/hold action
# dependent on action get the adj close price
# invest/sell a certain amount based on how much cash is available
# perform stocastic modeling similar to buy and hold to see how the investment changes for each different sectors in different time periods

# create function for signals 
def bb_band_roi(bb_signals_nd, adj_close_nd,periods_date,periods_list,tickers,n_sample,initial_investment):
    # create a nested dictionary with technical analysis signals and adj close price as pages
    all_data = {
        'Adj Close': adj_close_nd,
        'Bollinger Band': bb_signals_nd
    }

     # set the data that is going to be accessed
    adj_close_data = all_data['Adj Close']
    bollinger_band_data = all_data['Bollinger Band']

    # go through each period
    for period in periods_list:

        # set investment information
        cash = initial_investment

        # create the date range to be sampled from
        date_range = pd.date_range(start=pd.to_datetime(periods_date[period][0]),end=pd.to_datetime((periods_date[period][1]))-timedelta(days=90))

        # get the random dates for stochastic modeling
        start_dates = np.random.choice(date_range, size=n_sample, replace=False)

        for time_stamp in start_dates:
            time_stamp = pd.to_datetime(time_stamp)
             
            # get the data based on the start and end dates
            adj_close_period = adj_close_data[period].loc[time_stamp:time_stamp + timedelta(days=90)]
            bb_signals_period = bollinger_band_data[period].loc[time_stamp:time_stamp + timedelta(days=90)]

                # go through each day in the signals then collect the location
            for row_idx, row in bb_signals_period.iterrows():
                for col_idx,value in enumerate(row):
                    adj_close_price = adj_close_period.loc[row_idx, tickers[col_idx]]
                    if value=='Buy':

                        
                        print(f'during {period} purchase 5% in {tickers[col_idx]} at time {row_idx}')

                    elif value == 'Sell':
                        
                        print(f'during {period} sell 5% in {tickers[col_idx]} at time {row_idx}')

                    else: continue

        

bb_band_roi(bb_bands_signals,adj_close_sector_etf,economic_cycle_periods,economic_cycle_periods_list,sector_etf_tickers,1,1000)

# next step is to store the investment data and keep track of the portofolio
        

during trough purchase 5% in XLB at time 2009-03-02 00:00:00
during trough purchase 5% in XLI at time 2009-03-02 00:00:00
during trough purchase 5% in XLP at time 2009-03-02 00:00:00
during trough purchase 5% in XLE at time 2009-03-02 00:00:00
during trough purchase 5% in XLV at time 2009-03-02 00:00:00
during trough purchase 5% in XLP at time 2009-03-03 00:00:00
during trough purchase 5% in XLV at time 2009-03-03 00:00:00
during trough purchase 5% in VOX at time 2009-03-03 00:00:00
during trough purchase 5% in VOX at time 2009-03-09 00:00:00
during trough sell 5% in XLF at time 2009-03-18 00:00:00
during trough sell 5% in XLK at time 2009-03-18 00:00:00
during trough sell 5% in XLY at time 2009-03-18 00:00:00
during trough sell 5% in VOX at time 2009-03-18 00:00:00
during trough sell 5% in XLE at time 2009-03-19 00:00:00
during trough sell 5% in XLB at time 2009-03-23 00:00:00
during trough sell 5% in XLK at time 2009-03-23 00:00:00
during trough sell 5% in XLY at time 2009-03-23 00:0

In [22]:
all_data = {
    'Adj Close': sector_etf_adjusted_close,
    'Bollinger Band': bb_bands_signals
}

list = []

for period in economic_cycle_periods_list:
    for row_idx,row in all_data['Bollinger Band'][period].iterrows():
        for col_idx,value in enumerate(row):
            if value == 'Buy':
                adj_close_price = adj_close_sector_etf[period].loc[row_idx,sector_etf_tickers[col_idx]]
                print(f'buy ticker {sector_etf_tickers[col_idx]} at day {row_idx} for {adj_close_price}')
                
            elif value =='Sell':
                adj_close_price = adj_close_sector_etf[period].loc[row_idx,sector_etf_tickers[col_idx]]
                print(f'Sell ticker {sector_etf_tickers[col_idx]} at day {row_idx} for {adj_close_price}')
            else: continue

buy ticker XLF at day 2008-11-12 00:00:00 for 7.506924152374268
buy ticker XLY at day 2008-11-12 00:00:00 for 15.14069652557373
buy ticker XLI at day 2008-11-19 00:00:00 for 14.938098907470703
buy ticker XLF at day 2008-11-19 00:00:00 for 6.3178253173828125
buy ticker XLV at day 2008-11-19 00:00:00 for 18.3798885345459
buy ticker IYR at day 2008-11-19 00:00:00 for 15.563350677490234
buy ticker XLB at day 2008-11-20 00:00:00 for 13.991196632385254
buy ticker XLI at day 2008-11-20 00:00:00 for 14.53829574584961
buy ticker XLF at day 2008-11-20 00:00:00 for 5.639201641082764
buy ticker XLK at day 2008-11-20 00:00:00 for 10.49411678314209
buy ticker XLY at day 2008-11-20 00:00:00 for 13.221796989440918
buy ticker XLP at day 2008-11-20 00:00:00 for 14.213254928588867
buy ticker XLE at day 2008-11-20 00:00:00 for 23.99990463256836
buy ticker XLV at day 2008-11-20 00:00:00 for 17.856914520263672
buy ticker VOX at day 2008-11-20 00:00:00 for 24.479904174804688
buy ticker XLU at day 2008-11-20 

In [23]:
pd.Series(list).value_counts()

/var/folders/6q/lw13gkln44z1r6ncfrmcbc5w0000gn/T/ipykernel_94607/3774542663.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(list).value_counts()


Series([], dtype: int64)